In [29]:
#Libraries for the Project

import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate, GridSearchCV,RandomizedSearchCV
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import zscore

INFO 698 Notebook
Created 10/7/2025

Academic Data Analysis Capstone Project

In [ ]:
import pandas as pd
raw_data = pd.read_csv('DataSets/RawData.csv')

#print(raw_data.head())

Data Preparation:
We wil clean and modify the data set in preparation for bulding our models

In [13]:
#load dataset with null Term GPA excluded
data_set = pd.read_csv('DataSets/RawDataWithStudentsRemoved.csv')

print(data_set.head())

data_set.dtypes
data_set.shape


   FakeIdentifier  Age  Gender PrimaryMilitaryAffiliation AcademicCareer  \
0               1   18  Female    No Military Affiliation  Undergraduate   
1               2   72    Male    No Military Affiliation            Law   
2               3   26    Male    No Military Affiliation       Graduate   
3               4   47  Female                    Veteran       Graduate   
4               5   45    Male    No Military Affiliation  Undergraduate   

                          College  NumberofClassesEnrolled  TermGPA  \
0              College of Science                        4     3.75   
1   James E Rogers College of Law                        3     4.00   
2          College of Engineering                        2      NaN   
3       Coll of Ag Life & Env Sci                        2     4.00   
4  College of Information Science                        2     2.00   

   CumulativeGPA  UnitsPassedincludedinGPA  UnitsPassednotincludedinGPA  \
0          3.636                         

(66429, 15)

In [14]:

data_set_nulls = data_set.isnull().sum()
print(data_set_nulls)

FakeIdentifier                     0
Age                                0
Gender                             0
PrimaryMilitaryAffiliation         0
AcademicCareer                     0
College                            0
NumberofClassesEnrolled            0
TermGPA                        10860
CumulativeGPA                   3917
UnitsPassedincludedinGPA           0
UnitsPassednotincludedinGPA        0
UAFullTimePartTime                 0
AcademicLevelEndofTerm             0
FirstGenerationFlag                0
AcademicYear                       0
dtype: int64


In [15]:
data_set = data_set.dropna(subset = ['TermGPA'])



In [ ]:
#no nulls
data_set_nulls = data_set.isnull().sum()
print(data_set_nulls)

FakeIdentifier                 0
Age                            0
Gender                         0
PrimaryMilitaryAffiliation     0
AcademicCareer                 0
College                        0
NumberofClassesEnrolled        0
TermGPA                        0
CumulativeGPA                  0
UnitsPassedincludedinGPA       0
UnitsPassednotincludedinGPA    0
UAFullTimePartTime             0
AcademicLevelEndofTerm         0
FirstGenerationFlag            0
AcademicYear                   0
dtype: int64


In [21]:
#mean and standard dev by College

#mean_by_college = data_set.groupby('College')['TermGPA'].mean()
#print(mean_by_college)

stdev_by_college = data_set.groupby('College')['TermGPA'].std()
print(stdev_by_college)

College
Col Arch Plan & Landscape Arch    1.103857
Coll of Ag Life & Env Sci         1.182751
College of Applied Sci & Tech     1.261873
College of Education              0.926346
College of Engineering            1.058856
College of Fine Arts              1.260731
College of Humanities             1.274174
College of Information Science    1.144234
College of Medicine - Phoenix     0.000000
College of Medicine - Tucson      1.068167
College of Nursing                0.374531
College of Science                1.283719
College of Social & Behav Sci     1.286143
Colleges of Letters Arts & Sci    1.538084
Eller College of Management       1.185462
Graduate College                  1.026966
James C Wyant Coll Optical Sci    0.515825
James E Rogers College of Law     0.855457
Undergraduate Education           1.071179
Zuckerman Coll Public Health      0.981349
Name: TermGPA, dtype: float64


In [30]:
#add mean and standard dev by College to dataframe

data_set['meanTermGPACol'] = data_set.groupby('College')['TermGPA'].transform('mean')
data_set['stdevTermGPACol'] = data_set.groupby('College')['TermGPA'].transform('std')
data_set['ZTermGPA'] = data_set.groupby('College')['TermGPA'].transform(zscore)

print(data_set[['FakeIdentifier', 'TermGPA','College' ,'meanTermGPACol','stdevTermGPACol', 'ZTermGPA' ]].head())

   FakeIdentifier  TermGPA                         College  meanTermGPACol  \
0               1     3.75              College of Science        2.882671   
1               2     4.00   James E Rogers College of Law        3.470856   
3               4     4.00       Coll of Ag Life & Env Sci        3.081953   
4               5     2.00  College of Information Science        3.265459   
5               6     4.00   College of Social & Behav Sci        3.069600   

   stdevTermGPACol  ZTermGPA  
0         1.283719  0.675703  
1         0.855457  0.618709  
3         1.182751  0.776296  
4         1.144234 -1.105999  
5         1.286143  0.723453  


In [34]:
#create classes from normalized Term GPA
# very low z < -1.5
#low -1.5 to -0.5
#average -0.5 to 0.5
#above average 0.5 to 1.5
#very above avg > 1.5

#data_set['ClassTermGPA'] = ['WellBelowAverage' if 'ZTermGPA' < -1.5
                               #else 'BelowAverage' if ('ZTermGPA' < -0.5 & 'ZTermGPA' >= -1.5)
                              # else 'Average' if ('ZTermGPA' >=-0.5 & 'ZTermGPA' < 0.5)
                              # else 'AboveAverage' if 'ZTermGPA' >= 0.5 & 'ZTermGPA' < 1.5
                               #else 'WellAboveAverage' if 'ZtermGPA' >=1.5
                               #else 'NA']



data_set['ClassTermGPA'] = ['WellBelowAverage' if ztermgpa < -1.5
                               else 'BelowAverage' if (ztermgpa < -0.5 and ztermgpa >= -1.5)
                               else 'Average' if (ztermgpa >=-0.5 and ztermgpa < 0.5)
                               else 'AboveAverage' if ztermgpa >= 0.5 and ztermgpa < 1.5
                               else 'WellAboveAverage' if ztermgpa >=1.5
                               else 'NA' for ztermgpa in data_set['ZTermGPA']]

print(data_set[['ZTermGPA','ClassTermGPA']])

       ZTermGPA  ClassTermGPA
0      0.675703  AboveAverage
1      0.618709  AboveAverage
3      0.776296  AboveAverage
4     -1.105999  BelowAverage
5      0.723453  AboveAverage
...         ...           ...
66423  0.776296  AboveAverage
66424  0.572389  AboveAverage
66426  0.773971  AboveAverage
66427  0.229344       Average
66428 -0.069619       Average

[55569 rows x 2 columns]
